In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression, f_classif, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
from scipy.stats import pearsonr

path = Path('')



In [22]:
# Read the data into a Pandas dataframe
df = pd.read_parquet(path='../outage_data.parquet', engine='pyarrow')

# Remove duplicate entries in 2019
# Remove all rows with SimStartDate after 2019-01-01 and event_type == 'thunderstorm'
df = df.loc[~((df['SimStartDate'] > '2019-01-01') & (df['event_type'] == 'thunderstorm'))]
df.describe()

,breaker_counts,fuse_counts,switch_counts,transformer_counts,recloser_counts,pole_counts,grid_id,outage_count,x,y,...,canopy_interval_71_80_60,canopy_interval_81_90_60,canopy_interval_91_100_60,canopy_mean_60,canopy_median_60,canopy_var_60,length_proxy_30,line_length_30,length_proxy_60,line_length_60
count,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,7.515200e+04,...,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,7.515200e+04,7.515200e+04,7.515200e+04,7.515200e+04
mean,2.153689,93.809426,24.516393,328.192623,6.491803,1155.500000,243.500000,0.489940,63859.779114,1.198521e+06,...,0.106869,0.034790,0.000189,29.837696,28.834016,453.072222,2.942549e+05,4.720946e+06,4.973358e+05,4.720946e+06
std,4.329085,86.394655,34.519465,307.756527,7.954581,1075.414373,140.874107,1.356861,57254.344305,1.524949e+04,...,0.116302,0.064809,0.001851,19.703477,24.602453,263.134070,2.628181e+05,4.422564e+06,4.090464e+05,4.422564e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28082.000000,1.165209e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.160000e+02,0.000000e+00,1.106000e+03,0.000000e+00
25%,0.000000,21.000000,2.000000,64.000000,0.000000,288.000000,121.750000,0.000000,16918.000000,1.186893e+06,...,0.004917,0.000000,0.000000,11.905220,0.000000,251.758973,8.346350e+04,1.169986e+06,1.532600e+05,1.169986e+06
50%,0.000000,73.000000,11.000000,237.000000,3.000000,885.500000,243.500000,0.000000,52918.000000,1.201209e+06,...,0.062440,0.005809,0.000000,31.113005,26.000000,506.095082,2.260190e+05,3.405802e+06,4.117480e+05,3.405802e+06
75%,2.000000,141.250000,34.000000,512.500000,10.000000,1663.750000,365.250000,0.000000,103918.000000,1.210209e+06,...,0.173633,0.036777,0.000000,46.485635,47.000000,638.942526,4.424790e+05,7.083961e+06,7.661815e+05,7.083961e+06
max,36.000000,450.000000,281.000000,1291.000000,45.000000,4724.000000,487.000000,48.000000,262365.000000,1.237893e+06,...,0.500000,0.641975,0.037846,78.185185,83.000000,1305.809392,1.056230e+06,1.762393e+07,1.508426e+06,1.762393e+07


In [23]:
df.drop(columns = ['point_ewkt'], inplace = True)

# apply ordinal encoding to 'poly_ewkt', 'point_ewkt', 'event_type' columns
non_numerical_columns = ['poly_ewkt', 'event_type']

for column in non_numerical_columns:
    df[column] = df[column].astype('category')
    df[column] = df[column].cat.codes

# Convert datetime columns to separate columns for year, month, day, hour, minute, second
datetime_features = list(df.select_dtypes(include = "datetime64[ns, UTC]").columns)
for i in datetime_features:
    df[i+"_year"] = df[i].dt.year
    df[i+"_month"] = df[i].dt.month
    df[i+"_day"] = df[i].dt.day
    df[i+"_hour"] = df[i].dt.hour

# Use all data after Nov 1, 2018 (15 storms) as test set
train_df_total = df.loc[df['SimStartDate'] < '2018-11-01']
test_df_total = df.loc[df['SimStartDate'] >= '2018-11-01']

# Drop columns that are not needed for training
train_df = train_df_total.drop(datetime_features, axis=1)
test_df = test_df_total.drop(datetime_features, axis=1)

In [24]:
# Check the correlation between each feature and the target

lable = train_df['outage_count'].copy()

def check_correction(lable, feature):
    corr = lable.corr(train_df[feature])
    # print('The correlation between {} and the target is {:.4f}'.format(feature, corr))
    return abs(corr)

correlation_list = {}
for column in train_df.columns:
    if column != 'outage_count':
        # print(column)
        # print(train_df[column].dtype)
        correlation_list[column] = check_correction(lable, column)

# Select the features with a correlation than 0.8
# filtered_dict = {key: value for key, value in correlation_list.items() if value > 0.3}
correlation_list = dict(sorted(correlation_list.items(), key=lambda item: item[1], reverse=True))
# length = round(len(correlation_list) * 0.3)
length = 20
filtered_dict = dict(list(correlation_list.items())[0:length])
filtered_dict

{'transformer_counts': 0.3520876987746333,
 'fuse_counts': 0.34758017379124995,
 'pole_counts': 0.3450412160794607,
 'recloser_counts': 0.3399928217697246,
 'switch_counts': 0.28512049312449395,
 'breaker_counts': 0.25288776127800855,
 'poly_ewkt': 0.20830692702316497,
 'grid_id': 0.20830692702316497,
 'x': 0.1547943587772196,
 'Wind speed (gust)_m s**-1 (max)_lambert_level 10 m_gt_27': 0.13036606743638585,
 'y': 0.1190582548646127,
 'Surface pressure_Pa (instant)_lambert_level 0-filtered_16-max_min': 0.11609646099549441,
 'Wind speed (gust)_m s**-1 (max)_lambert_level 10 m_gt_22': 0.11524824011094276,
 'Wind speed (gust)_m s**-1 (max)_lambert_level 10 m_gt_17': 0.10036241444925928,
 'Wind speed (gust)_m s**-1 (max)_lambert_level 10 m_gt_44': nan,
 '220_220 (max)_lambert_levels 0-40000-filtered_32-mean_sum': 0.09871124330866496,
 '220_220 (max)_lambert_levels 0-40000-filtered_32-mean_mean': 0.09871124330866496,
 '221_221 (min)_lambert_levels 0-40000-filtered_2-mean_sum': 0.097802581261

In [33]:
Feature_list = ['breaker_counts',
'fuse_counts',
'switch_counts',
'transformer_counts',
'recloser_counts',
'pole_counts',
'poly_ewkt',
# 'point_ewkt',
'grid_id',
# 'SimStartDate',
# 'outage_start_time',
# 'outage_end_time',
# 'weather_start_time',
# 'weather_end_time',
'event_type',
'x',
'y',]

time_features = ['SimStartDate_year', 'SimStartDate_month',
'SimStartDate_day', 'SimStartDate_hour', 'outage_start_time_year',
'outage_start_time_month', 'outage_start_time_day',
'outage_start_time_hour', 'outage_end_time_year',
'outage_end_time_month', 'outage_end_time_day', 'outage_end_time_hour',
'weather_start_time_year', 'weather_start_time_month',
'weather_start_time_day', 'weather_start_time_hour',
'weather_end_time_year', 'weather_end_time_month',
'weather_end_time_day', 'weather_end_time_hour']

Feature_list = Feature_list + list(filtered_dict.keys())
# Feature_list = Feature_list + time_features + list(filtered_dict.keys())
Feature_list = list(set(Feature_list))
print(len(Feature_list))

21


In [37]:
import pickle   
Feature_list = pickle.load(open('drop_list_group.pkl', 'rb'))
droplst = ['point_ewkt', 'SimStartDate', 'outage_start_time', 'outage_end_time', 'weather_start_time', 'weather_end_time']
Feature_list = [x for x in Feature_list if x not in droplst]
print(len(Feature_list))

1051


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OrdinalEncoder


train_X = train_df[Feature_list]; test_X = test_df[Feature_list]
train_y = train_df['outage_count']; test_y = test_df['outage_count']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_X)
X_test_scaled = scaler.transform(test_X)

model = LinearRegression()
# model = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)


model.fit(X_train_scaled, train_y)

test_preds = model.predict(X_test_scaled)

In [39]:
# Aggregate predictions by SimStartDate and event_type
preds_df = test_X.copy()
preds_df['outage_count'] = test_preds
preds_df['SimStartDate'] = test_df_total['SimStartDate']

grouped_preds = preds_df.groupby(['SimStartDate','event_type']).outage_count.sum().sort_index()
print("Number of predictions: ", len(grouped_preds))
display(grouped_preds)

# Aggregate true values by SimStartDate and event_type
true_vals_df = test_X.copy()
true_vals_df['outage_count'] = test_y
true_vals_df['SimStartDate'] = test_df_total['SimStartDate']

grouped_trues = true_vals_df.groupby(['SimStartDate','event_type']).outage_count.sum().sort_index()
print("Number of true values: ", len(grouped_trues))
display(grouped_trues)

mse_sq = np.square(grouped_preds - grouped_trues).mean()
print("MSE: ", mse_sq)

mae = np.abs(grouped_preds - grouped_trues).mean()
print("MAE: ", mae)

mape = np.abs((grouped_preds - grouped_trues) / grouped_trues).mean()
print("MAPE: ", mape)


Number of predictions:  15


SimStartDate               event_type
2018-11-03 00:00:00+00:00  0             313.0
2018-11-06 09:00:00+00:00  0             157.0
2018-11-09 21:00:00+00:00  0             199.0
2018-11-13 03:00:00+00:00  0             137.0
2018-11-24 09:00:00+00:00  0             184.0
2018-11-26 09:00:00+00:00  0             167.0
2018-11-28 09:00:00+00:00  0             132.0
2018-12-15 06:00:00+00:00  0              85.0
2018-12-17 06:00:00+00:00  0             169.0
2018-12-20 21:00:00+00:00  0             448.0
2018-12-28 09:00:00+00:00  0             124.0
2019-01-20 15:00:00+00:00  0             262.0
2019-01-23 21:00:00+00:00  0             435.0
2019-01-30 09:00:00+00:00  0             240.0
2019-02-24 09:00:00+00:00  0             656.0
Name: outage_count, dtype: float64

Number of true values:  15


SimStartDate               event_type
2018-11-03 00:00:00+00:00  0             313.0
2018-11-06 09:00:00+00:00  0             157.0
2018-11-09 21:00:00+00:00  0             199.0
2018-11-13 03:00:00+00:00  0             137.0
2018-11-24 09:00:00+00:00  0             184.0
2018-11-26 09:00:00+00:00  0             167.0
2018-11-28 09:00:00+00:00  0             132.0
2018-12-15 06:00:00+00:00  0              85.0
2018-12-17 06:00:00+00:00  0             169.0
2018-12-20 21:00:00+00:00  0             448.0
2018-12-28 09:00:00+00:00  0             124.0
2019-01-20 15:00:00+00:00  0             262.0
2019-01-23 21:00:00+00:00  0             435.0
2019-01-30 09:00:00+00:00  0             240.0
2019-02-24 09:00:00+00:00  0             656.0
Name: outage_count, dtype: float64

MSE:  6.086239850800324e-23
MAE:  6.859105875870834e-12
MAPE:  3.1948883075235884e-14


In [ ]:
feature = pd.read_csv('../features.csv')
feature.sort_values(by='0', ascending=False).to_csv('../features_sorted.csv', index=False)
